In [1]:
!pip install pulp
import pandas as pd
import pulp
import numpy as np
import datetime
import matplotlib.pyplot as plt 

In [2]:
df_distance = pd.read_csv("distances.csv")
df_drivers_full = pd.read_csv("part3_drivers.csv")
df_3 = pd.read_csv("part3_small.csv")

start_time = pd.to_datetime('2022-04-02 17:00:00')
W = 30
W_range = range(0, 50, 5)

## Only one driver

In [3]:
df_drivers_full 

,start region,velocity
0,Downtown Toronto (Richmond / Adelaide / King),40
1,Downtown Toronto (St. James Park),35
2,Downtown Toronto (Church and Wellesley),32


In [4]:
df_drivers = df_drivers_full.loc[[0],:]

In [5]:
df_drivers

,start region,velocity
0,Downtown Toronto (Richmond / Adelaide / King),40


In [6]:
df_3

,restaurant,customer,estimated availability
0,Downtown Toronto (Central Bay Street),North York (Armour Heights / Wilson Heights / ...,2022-04-02 5:17 PM
1,Downtown Toronto (Christie),West Toronto (Brockton / Parkdale Village / Ex...,2022-04-02 5:49 PM
2,Downtown Toronto (Ryerson),York (Fairbank / Oakwood),2022-04-02 6:17 PM
3,Downtown Toronto (St. James Park),East Toronto (The Beaches),2022-04-02 5:53 PM
4,Downtown Toronto (Kensington Market / Chinatow...,Downtown Toronto (Central Bay Street),2022-04-02 5:24 PM


## Sets

In [7]:
vehicle = [0 ]
vehicle

[0]

In [8]:
start_region = []
for v in vehicle:
    start_region.append(df_drivers["start region"][v])
start_region

['Downtown Toronto (Richmond / Adelaide / King)']

In [9]:
restaurant = df_3["restaurant"].unique().tolist()
restaurant

['Downtown Toronto (Central Bay Street)',
 'Downtown Toronto (Christie)',
 'Downtown Toronto (Ryerson)',
 'Downtown Toronto (St. James Park)',
 'Downtown Toronto (Kensington Market / Chinatown / Grange Park)']

In [10]:
customer = df_3["customer"].unique().tolist()
customer

['North York (Armour Heights / Wilson Heights / Downsview North)',
 'West Toronto (Brockton / Parkdale Village / Exhibition Place)',
 'York (Fairbank / Oakwood)',
 'East Toronto (The Beaches)',
 'Downtown Toronto (Central Bay Street)']

In [11]:
loc_index = {}
locations = start_region + restaurant + customer

for i, location in enumerate(locations):
    loc_index[i] = location
loc_index

{0: 'Downtown Toronto (Richmond / Adelaide / King)',
 1: 'Downtown Toronto (Central Bay Street)',
 2: 'Downtown Toronto (Christie)',
 3: 'Downtown Toronto (Ryerson)',
 4: 'Downtown Toronto (St. James Park)',
 5: 'Downtown Toronto (Kensington Market / Chinatown / Grange Park)',
 6: 'North York (Armour Heights / Wilson Heights / Downsview North)',
 7: 'West Toronto (Brockton / Parkdale Village / Exhibition Place)',
 8: 'York (Fairbank / Oakwood)',
 9: 'East Toronto (The Beaches)',
 10: 'Downtown Toronto (Central Bay Street)'}

In [12]:
start = np.arange(0, len(loc_index)).tolist()
start

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [13]:
end = start[len(start_region):]
end

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [14]:
start_regions = start[:len(start_region)]
start_regions

[0]

In [15]:
restaurants = start[len(start_region):-len(customer)]
restaurants

[1, 2, 3, 4, 5]

In [16]:
customers = start[-len(customer):]
customers

[6, 7, 8, 9, 10]

In [17]:
step = np.arange(0, len(locations)).tolist()
step

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [18]:
list_order = []
for index, row in df_3.iterrows():
    list_order.append([row["restaurant"], row["customer"]])
print(list_order)

[['Downtown Toronto (Central Bay Street)', 'North York (Armour Heights / Wilson Heights / Downsview North)'], ['Downtown Toronto (Christie)', 'West Toronto (Brockton / Parkdale Village / Exhibition Place)'], ['Downtown Toronto (Ryerson)', 'York (Fairbank / Oakwood)'], ['Downtown Toronto (St. James Park)', 'East Toronto (The Beaches)'], ['Downtown Toronto (Kensington Market / Chinatown / Grange Park)', 'Downtown Toronto (Central Bay Street)']]


## Parameters

In [19]:
restaurant_index = {}
for key in restaurants:
    for value in restaurant:
        restaurant_index[key] = value
        restaurant.remove(value)
        break  

In [20]:
restaurant_index

{1: 'Downtown Toronto (Central Bay Street)',
 2: 'Downtown Toronto (Christie)',
 3: 'Downtown Toronto (Ryerson)',
 4: 'Downtown Toronto (St. James Park)',
 5: 'Downtown Toronto (Kensington Market / Chinatown / Grange Park)'}

In [21]:
customer_index = {}
for key in customers:
    for value in customer:
        customer_index[key] = value
        customer.remove(value)
        break  

In [22]:
restaurant_index

{1: 'Downtown Toronto (Central Bay Street)',
 2: 'Downtown Toronto (Christie)',
 3: 'Downtown Toronto (Ryerson)',
 4: 'Downtown Toronto (St. James Park)',
 5: 'Downtown Toronto (Kensington Market / Chinatown / Grange Park)'}

In [23]:
location_distance = {}
for i in start:
    for j in end: 
        if loc_index[i] == loc_index[j]:
            location_distance[(i,j)] = 0 
        else:
            location_distance[(i,j)] = df_distance[(df_distance["origin"] == loc_index[i]) &(df_distance["destination"] == loc_index[j])]["distance"].item()
            

In [24]:

min_arrival_time_mins = {}
for j in end:
    if j in restaurants:
        delta = pd.to_datetime(df_3.loc[df_3["restaurant"] == loc_index[j]]["estimated availability"].tolist()[0]) - start_time
        min_arrival_time_mins[j] = delta/datetime.timedelta(minutes=1)
    if j in customers: 
        min_arrival_time_mins[j] = 0 
min_arrival_time_mins
#0,1,2 are start regions

{1: 17.0, 2: 49.0, 3: 77.0, 4: 53.0, 5: 24.0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0}

In [25]:
drivers_start = {}
for v in vehicle:
    drivers_start[v] = df_drivers.iloc[v][0]
drivers_start

{0: 'Downtown Toronto (Richmond / Adelaide / King)'}

In [26]:
drivers_velocity = {}
for v in vehicle:
    drivers_velocity[v] = 60 /df_drivers.iloc[v][1] # convert to minutes / km 
drivers_velocity  

{0: 1.5}

In [27]:
#Calculate travel time for each distance between i and j
wait_time = 5

travel_time = {}
for i in start:
    for j in end:
        for v in vehicle:
            if i == j:
                travel_time[(i,j,v)] = 0 
            else: 
                if i in customers:
                    travel_time[(i,j,v)] = location_distance[(i,j)]*drivers_velocity[v] + wait_time
                else:
                    travel_time[(i,j,v)] = location_distance[(i,j)]*drivers_velocity[v] 

### Variables

In [28]:
xvar = pulp.LpVariable.dict("x", (start, end, vehicle, step), cat=pulp.LpBinary)
dvar = pulp.LpVariable.dict("d", (vehicle, step), lowBound=0.0, cat=pulp.LpContinuous)
wvar = pulp.LpVariable.dict("w", (end, vehicle, step), lowBound=0.0, cat=pulp.LpContinuous)
zvar = pulp.LpVariable.dict("z", (vehicle, step), lowBound=0.0, cat=pulp.LpContinuous)

In [29]:
drivers_start

{0: 'Downtown Toronto (Richmond / Adelaide / King)'}

In [30]:
def get_key_from_driver_start(val):
    for key, value in drivers_start.items():
         if val == value:
                return key           
get_key_from_driver_start(drivers_start[0])

0

In [31]:
for v in vehicle:
    print(drivers_start[v])
    

Downtown Toronto (Richmond / Adelaide / King)


### Model & Constraints

In [32]:
flowmodel = pulp.LpProblem(name = "FlowModel", sense = pulp.LpMinimize)

# Constraint 1: every location visited once by one vehicle
for j in end: 
    flowmodel += pulp.lpSum([xvar[(i,j,v,t)] for i in start for v in vehicle for t in step]) == 1

# #Constraint 2: Force every vehicle takes at least one order:
# for v in vehicle:
#     flowmodel += pulp.lpSum([xvar[i,j,v,t]] for i in start for j in end for t in step) >= 1

# Constraint 3: Conservation of flow
for v in vehicle:  
    for t in step[:-1]:
        for j in end:
            flowmodel += (pulp.lpSum([xvar[(i,j,v,t)] for i in start]) >= pulp.lpSum([xvar[(j,k,v,t+1)] for k in end]))

# Constraint 4: Each driver starts at their own region and has an outflow of 1 in the first step, others are zero
for v in vehicle:
    flowmodel += pulp.lpSum(xvar[(v,j,v,0)] for j in end) == 1
    flowmodel += pulp.lpSum(xvar[(j,j,v,0)] for j in end) == 0
    # each driver only starts from their starting region once
    flowmodel += pulp.lpSum(xvar[(v,j,v,t)] for j in end for t in step[1:]) == 0
    # each driver cannot start from other locations
    vehicle_copy = vehicle.copy()
    vehicle_copy.remove(v)
    flowmodel += pulp.lpSum(xvar[(a,j,v,t)] for a in vehicle_copy for j in end for t in step[1:]) == 0

        
# Constraint 5: for each vehicle at each time step, there should be only 1 xvar <= 1 (=0 if nothing happened within this time block)
for v in vehicle:
    for t in step:
        flowmodel += pulp.lpSum(xvar[(i,j,v,t)] for i in start for j in end) <= 1
        
# Constraint 6: make sure driver goes to a restaurant before delivering the food
def get_key_from_restaurant(val):
    for key, value in restaurant_index.items():
         if val == value:
                return key
            
def get_key_from_customer(val):
    for key, value in customer_index.items():
         if val == value:
                return key
    
def find_restaurant_of_customer(customer):
    for order in list_order:
        if customer == get_key_from_customer(order[1]):
            return get_key_from_restaurant(order[0])
        
for t in step: 
    for v in vehicle:
        for j in end: 
            if j in customers: 
                flowmodel += pulp.lpSum( [xvar[i,j,v,t] for i in start]) <= pulp.lpSum([xvar[i, find_restaurant_of_customer(j),v, t_] for i in start for t_ in step[:t]])

# # Constraint 7: Time to arrive at the location of the t-th step
for v in vehicle:
    for t in step:
        if t == 0: 
            flowmodel += dvar[v,t] >= pulp.lpSum([ xvar[(i,j,v,0)] * min_arrival_time_mins[j] for i in start for j in end])
            flowmodel += dvar[v,t] >= pulp.lpSum([ xvar[(i,j,v,t)] * travel_time[(i,j,v)] for i in start for j in end])
        else:
            flowmodel += dvar[v,t] >= (dvar[v,t-1] + pulp.lpSum([ xvar[(i,j,v,t)] * travel_time[(i,j,v)] for i in start for j in end]))      

# # Constraint 8: if arriving at location j, ensures time is at least the min_arrival_time_mins
for t in step:
    for i in start:
        for j in end:
            for v in vehicle:
                flowmodel += dvar[v,t] >= min_arrival_time_mins[j] * xvar[i,j,v,t]


# Constraint 9: ensures the average waiting time of the orders is lower than the maximum W
# implementation idea: 
#         flowmodel += pulp.lpSum([wvar[j,t] * xvar[i,j,t] for i in start]) <= W
# use zvar[t] with big Constant M to denote wvar[j,t] * xvar[i,j,t] as two decision variables cannot be multiplied together
# zvar[t] is the total waiting time up until the step t
# Constraint: define waiting time of each order at step t: wvar[j,t]

for v in vehicle:
    for t in step:
        for j in end:
            if j in restaurants:
                flowmodel += wvar[(j,v,t)]  == 0
            else:
                flowmodel += wvar[(j,v,t)] == dvar[v,t] - pulp.lpSum([ min_arrival_time_mins[find_restaurant_of_customer(j)] * xvar[(i,j,v,t)] for i in start])


M = 100000
for t in step:
    for i in start:
        for j in end:
            for v in vehicle:
                if t == 0:
                    flowmodel += zvar[v,t] == 0 
                elif t == 1:
                    flowmodel += zvar[v,t] >= wvar[j,v,t] - M * (1 - xvar[(i,j,v,t)])
                    flowmodel += zvar[v,t] <= wvar[j,v,t] + M * (1 - xvar[(i,j,v,t)])
                else: 
                    flowmodel += zvar[v,t] >= zvar[v,t-1] + wvar[(j,v,t)] - M * (1 - xvar[(i,j,v,t)])
                    flowmodel += zvar[v,t] <= zvar[v,t-1] + wvar[(j,v,t)] + M * (1 - xvar[(i,j,v,t)])

                
for t in step:
    flowmodel += pulp.lpSum(zvar[v,t] for v in vehicle) <= W * len(customers)
 

### Objective Functions

In [33]:
obj = pulp.lpSum([location_distance[(i,j)] * xvar[(i,j,v,t)] for i in start for j in end for v in vehicle for t in step])
flowmodel += obj

### Solution Process

In [34]:
flowmodel.solve()
print("Status:", pulp.LpStatus[flowmodel.status])

Status: Optimal


In [35]:
# Total Distance
totalDistance =pulp.value(flowmodel.objective)
print("Total Distance: " + str(totalDistance))

Total Distance: 51.06492929750579


In [58]:
# print solution  
for t in step: 
    for v in vehicle:
        print("Step: " + str(t))
        print("Total travel time (in minutes) up to this step: " + str(dvar[v,t].varValue))
    
    for v in vehicle:
        for i in start: 
            for j in end: 

                if xvar[(i,j,v,t)].varValue > 0.0:
                    print("\tVehicle: " + str(v))
                    print("\tLeave from " + str(i) + ": " + str(loc_index.get(i)))
                    print("\tTravel to " + str(j) + ": " + str(loc_index.get(j)))

                    print("\tWaiting time: " + str(wvar[(j,v,t)].varValue))
                    print("\tTotal waiting time up to this step: " + str(zvar[v,t].varValue))

print("Average waiting time: " + str(total_waiting_time/len(customers)) )


Step: 0
Total travel time (in minutes) up to this step: 53.0
	Vehicle: 0
	Leave from 0: Downtown Toronto (Richmond / Adelaide / King)
	Travel to 4: Downtown Toronto (St. James Park)
	Waiting time: 0.0
	Total waiting time up to this step: 0.0
Step: 1
Total travel time (in minutes) up to this step: 63.822594
	Vehicle: 0
	Leave from 4: Downtown Toronto (St. James Park)
	Travel to 9: East Toronto (The Beaches)
	Waiting time: 0.0
	Total waiting time up to this step: 17.79745
Step: 2
Total travel time (in minutes) up to this step: 79.579157
	Vehicle: 0
	Leave from 9: East Toronto (The Beaches)
	Travel to 3: Downtown Toronto (Ryerson)
	Waiting time: 0.0
	Total waiting time up to this step: 17.79745
Step: 3
Total travel time (in minutes) up to this step: 81.988452
	Vehicle: 0
	Leave from 3: Downtown Toronto (Ryerson)
	Travel to 5: Downtown Toronto (Kensington Market / Chinatown / Grange Park)
	Waiting time: 0.0
	Total waiting time up to this step: 28.301808
Step: 4
Total travel time (in minute

In [37]:
df_3

,restaurant,customer,estimated availability
0,Downtown Toronto (Central Bay Street),North York (Armour Heights / Wilson Heights / ...,2022-04-02 5:17 PM
1,Downtown Toronto (Christie),West Toronto (Brockton / Parkdale Village / Ex...,2022-04-02 5:49 PM
2,Downtown Toronto (Ryerson),York (Fairbank / Oakwood),2022-04-02 6:17 PM
3,Downtown Toronto (St. James Park),East Toronto (The Beaches),2022-04-02 5:53 PM
4,Downtown Toronto (Kensington Market / Chinatow...,Downtown Toronto (Central Bay Street),2022-04-02 5:24 PM
